Testing Environment

In [7]:
import sys
import os
cwd, _ = os.path.split("__file__")
parent_dir = os.path.join(cwd, "..")
os.environ["BPZDATAPATH"] = os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files")
filt = os.path.join(os.path.expandvars("$LEPHAREWORK"), "filt")
lib_bin = os.path.join(os.path.expandvars("$LEPHAREWORK"), "lib_bin")
lib_mag = os.path.join(os.path.expandvars("$LEPHAREWORK"), "lib_mag")
output_paths = [filt, lib_bin, lib_mag]
for path in output_paths:
    if not os.path.isdir(path):
        os.mkdir(path)

## PyLePHARE Tests

In [2]:
import pandas
from pylephare import io
data = pandas.read_csv(io._PACKAGE_ROOT+"/data/data_test.csv", sep=" ")
configfile = None # default file stored in your $LEPHAREWORK/pylephare/config/default.config

In [3]:
print(data)

         sdss.u    sdss.u.err         ps1.g     ps1.g.err         ps1.r  \
0  1.227194e-17  4.928681e-18  1.389326e-17  6.902862e-20  8.659294e-18   
1  3.189998e-17  4.026194e-18  7.381098e-17  1.799055e-19  7.192829e-17   
2  6.248074e-16  7.136033e-18  1.801086e-15  1.164064e-18  2.002177e-15   

      ps1.r.err         ps1.i     ps1.i.err         ps1.z     ps1.z.err  \
0  2.922806e-20  9.346882e-18  4.667556e-20  3.230372e-18  2.169672e-20   
1  9.228955e-20  8.032923e-17  1.650248e-19  6.790159e-17  9.719448e-20   
2  4.736618e-19  2.109367e-15  8.014284e-19  1.743491e-15  4.585604e-19   

   CONTEXT   Z-SPEC       STRING  
0       31  0.03185       2007aj  
1       31  0.03509        2002G  
2       31  0.02824  ASASSN-15la  


In [4]:
from pylephare import lephare
lp = lephare.LePhare(data, configfile)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/allake/code/roman_filters/py/../LEPHARE/filt/ps1/g_ps.pb'

In [ ]:
fileouts = lp.run()

In [ ]:
lp.config.get_config("configfile")

## Monte Carlo Fit

In [ ]:
data = pandas.read_csv(io._PACKAGE_ROOT+"/data/data_test.csv", sep=" ")
target = data.iloc[0]

configfile = None # use the default one

In [ ]:
from pylephare import montecarlo
mclp = montecarlo.MCLePhare(target, 50)

In [ ]:
mclp.load_lephare(configfile)

In [ ]:
mclp.lephare.config.set_intrinsic_error([0.1,0.08,0.08,0.08,0.1])

In [ ]:
mclp.run()

In [ ]:
mclp.show()

In [ ]:
mclp.catout

In [ ]:
mclp.get_synthetic_photometry('sdss.u', restframe=True, influx=False)

In [21]:
cwd = os.path.abspath(os.path.split("__file__")[0])
parent_dir = os.path.join(cwd, "..")
# os.environ["LEPHAREDIR"] = os.path.join(parent_dir, "LEPHARE")
# os.environ["LEPHAREWORK"] = os.path.join(cwd)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
data_dir = os.environ.get("BPZDATAPATH", None)
fil_dir = None if data_dir is None else os.path.join(data_dir, "FILTER")
ab_dir = None if data_dir is None else os.path.join(data_dir, "AB")
sed_dir = None if data_dir is None else os.path.join(data_dir, "SED")
paths = [cwd, parent_dir, os.environ.get("BPZDATAPATH", None)]
for path in paths:
    print(path)

/Users/allake/code/roman_filters/py
/Users/allake/code/roman_filters/py/..
/Users/allake/code/roman_filters/DESC_BPZ/src/desc_bpz/data_files


## BPZ Tests

In [3]:
import desc_bpz

../DESC_BPZ/src/desc_bpz/data_files


In [14]:
import inspect

names = dir(desc_bpz)

for name in names:
    obj = getattr(desc_bpz, name)
    if inspect.isfunction(obj):
        print(name)

In [15]:
members = inspect.getmembers(desc_bpz)

functions = [m for m in members if inspect.isfunction(m[1])]

for name, obj in functions:
    print(name)

In [4]:
import numpy as np
from desc_bpz.bpz_tools_py3 import prior_with_dict, prior

def test_old_vs_new_prior():
    zgrid = np.arange(0, 10.01, 0.1)
    mags = np.array([20.5, 21., 22.])
    # copy params needed for hdfn_gen prior
    paramdict = dict(fo_arr=[0.35, 0.5], kt_arr=[0.45, 0.147],
                     alpha_arr=[2.465, 1.806, 0.906],
                     zo_arr=[0.431, 0.390, 0.0626],
                     km_arr=[0.0913, 0.0636, 0.123],
                     m0=20.0,
                     numpertype=[1, 2, 3])

    for mag in mags:
        oldprior = prior(zgrid, mag, 'hdfn_gen')
        newprior = prior_with_dict(zgrid, mag, paramdict)
        assert (oldprior == newprior).all()
test_old_vs_new_prior()

In [10]:
import os
import pytest
# dirname, _ = os.path.split(__file__)
# parent = os.path.join(dirname, '../src/desc_bpz/data_files/')


# This test has to go first 
def test_no_path():
    tmp = os.environ.get("BPZDATAPATH")
    if tmp is not None:
        del os.environ["BPZDATAPATH"]

    import desc_bpz.paths

    # Future proof in case a later test import the
    # module and are called first
    if desc_bpz.paths.data_dir is not None:
        # In this case first restore the env var
        if tmp is not None:
            os.environ["BPZDATAPATH"] = tmp
        # then mark the test as skipped
        pytest.skip("desc_bpz already imported by another test and path set")

    # should fail if BPZDATAPATH not set and we have not set manually
    with pytest.raises(RuntimeError):
        print(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))

    # restore env var
    if tmp is not None:
        os.environ["BPZDATAPATH"] = tmp


def test_path():
    import desc_bpz.paths
    data_dir = os.environ.get("BPZDATAPATH", None)
    fil_dir = None if data_dir is None else data_dir + "/FILTER/"
    ab_dir = None if data_dir is None else data_dir + "/AB/"
    sed_dir = None if data_dir is None else data_dir + "/SED/"
    print(os.environ.get("BPZDATAPATH"))
    assert os.path.exists(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))
    assert os.path.exists(desc_bpz.paths.get_fil_file("DC2LSST_g.res"))
    assert os.path.exists(desc_bpz.paths.get_sed_file("CWWSB4.list"))


test_no_path()
test_path()


Skipped: desc_bpz already imported by another test and path set

In [8]:
import os
import git

base_dir = os.path.abspath(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(base_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
path = "$BPZDATAPATH"
print(os.path.expandvars(path))

/Users/allake/code/roman_filters/DESC_BPZ/src/desc_bpz/data_files
